In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import sys
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.append(path)
import pandas as pd
from utils.helpers import is_question_arabic
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import joblib

c:\Users\nasser\.conda\envs\finall\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('../data/raw.csv')

In [3]:
df = df[['message']]

In [4]:
# remove null values
df = df.dropna()

# Label data using Regex

In [5]:
df['is_question'] = df['message'].apply(lambda x : 1 if is_question_arabic(x) else 0)

# Train NN model

In [6]:
messages = df['message'].values
labels = df['is_question'].values
vocab_size = 10000 # Choose a vocabulary size
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(messages)
sequences = tokenizer.texts_to_sequences(messages)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=100)
y = tf.keras.utils.to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Define RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [45]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [46]:
# Train model and prevent overfitting with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, batch_size=2048, validation_split=0.2)


Epoch 1/20
31/31 [==============================] - 3s 55ms/step - loss: 0.6227 - accuracy: 0.6449 - val_loss: 0.4528 - val_accuracy: 0.8022
Epoch 2/20
31/31 [==============================] - 1s 43ms/step - loss: 0.2738 - accuracy: 0.8953 - val_loss: 0.1515 - val_accuracy: 0.9507
Epoch 3/20
31/31 [==============================] - 1s 43ms/step - loss: 0.1747 - accuracy: 0.9477 - val_loss: 0.1347 - val_accuracy: 0.9555
Epoch 4/20
31/31 [==============================] - 1s 43ms/step - loss: 0.0906 - accuracy: 0.9729 - val_loss: 0.1099 - val_accuracy: 0.9658
Epoch 5/20
31/31 [==============================] - 1s 43ms/step - loss: 0.0669 - accuracy: 0.9815 - val_loss: 0.1088 - val_accuracy: 0.9653
Epoch 6/20
31/31 [==============================] - 1s 43ms/step - loss: 0.0569 - accuracy: 0.9849 - val_loss: 0.1139 - val_accuracy: 0.9644
Epoch 7/20
31/31 [==============================] - 1s 42ms/step - loss: 0.0522 - accuracy: 0.9860 - val_loss: 0.1071 - val_accuracy: 0.9696
Epoch 8/20
31

In [47]:
# Evaluate model on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss on test set: {loss}")
print(f"Accuracy on test set: {accuracy}")

# # Save model and tokenizer
# model.save('..\models\question_classifier.h5')
# joblib.dump(tokenizer, '..\models\question_tokenizer.joblib')

602/602 [==============================] - 4s 6ms/step - loss: 0.1244 - accuracy: 0.9686
Loss on test set: 0.1243947222828865
Accuracy on test set: 0.9685844779014587


In [48]:
from sklearn.metrics import classification_report

# Assuming you have predictions for the test set
y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_labels = [np.argmax(pred) for pred in y_pred]
y_test_labels = [np.argmax(true) for true in y_test]

# Generate the classification report
class_report = classification_report(y_test_labels, y_pred_labels)

print(class_report)


602/602 [==============================] - 3s 4ms/step
              precision    recall  f1-score   support

           0       0.97      0.98      0.97     11340
           1       0.97      0.96      0.96      7918

    accuracy                           0.97     19258
   macro avg       0.97      0.97      0.97     19258
weighted avg       0.97      0.97      0.97     19258



In [49]:
# create dataframa with text and predict if it is a question or not
new_df = pd.DataFrame({'message': ['شو الاخبار ']})
# add more rows

# tokenize the text
new_sequences = tokenizer.texts_to_sequences(new_df['message'].values)
# pad the sequences
new_X = tf.keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=100)
# predict
model.predict(new_X)


1/1 [==============================] - 0s 17ms/step


array([[3.8270721e-07, 9.9999964e-01]], dtype=float32)